In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='en')

In [3]:
headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'host': 'careers.microsoft.com',
    'referer': 'http://careers.microsoft.com/us/en'
}

In [4]:
URL = "http://careers.microsoft.com/us/en/search-results?s=1"

In [5]:
result = requests.get(URL, headers=headers)
result

<Response [200]>

In [6]:
soup = BeautifulSoup(result.text)

In [9]:
print(soup.find('script'))

<script type="text/javascript"> /*<!--*/ var phApp = phApp || {"widgetApiEndpoint":"https://careers.microsoft.com/widgets","country":"us","deviceType":"desktop","locale":"en_us","absUrl":true,"refNum":"MICRUS","cdnUrl":"https://prodcmscdn.azureedge.net/careerconnectresources/p","baseUrl":"https://careers.microsoft.com/us/en/","baseDomain":"https://careers.microsoft.com","phenomTrackURL":"careers.microsoft.com/us/en/phenomtrack.min.js","pageName":"search-results","siteType":"external","rootDomain":"https://careers.microsoft.com","pageId":"page19"}; phApp.ddo = {"siteConfig":{"status":"success","errorCode":null,"errorMsg":null,"data":{"urlMap":{"home":"home","category":"c/:category","job":"job/:jobSeqNo/:title","jobcart":"jobcart","search-results":"search-results","glassdoor-reviews":"glassdoor-reviews"},"categoryUrlMap":{"MICRUS_Legal---Corporate-Affairs":"c/legal-corporate-affairs-jobs","MICRUS_Supply-Chain---Operations-Management":"c/supply-chain-operations-management-jobs","MICRUS_Fi

здесь в phApp.ddo есть описания вакансий, но нужено парсить джаваскрипт... конкретнее нас инетерсует ключ eagerLoadRefineSearch внутри оson

In [10]:
pattern = r'""eagerLoadRefineSearch".+?({.+}).+?"eid"'

In [15]:
re.search(pattern, soup.find('script').text)